In [20]:
# clearing the memory
rm(list = ls())

In [21]:
# installing pacman
if (require(pacman) == F) install.packages('pacman')
library(pacman)

In [3]:
# libraries
p_load(caret, doParallel, kernlab, randomForest, nnet, 
       xgboost, foreach, e1071, pROC, EMP)

In [26]:
cd <- 'C:/Users/PC/Fair_Credit_Scoring-main/codes'

In [27]:
setwd(cd)

In [28]:
# paths
source(file.path(cd, 'code_00_working_paths.R'))

In [29]:
# data 
data <- 'taiwan'

In [33]:
# partitioning
num_folds <- 5
seed      <- 1

# cores
cores <- 8

# repair level
all_di_repair_level <- c('0.5', '0.6', '0.7', '0.8', '0.9', '1.0')

# options
set.seed(seed)
options(scipen = 10)

In [34]:
# parallel computing
nrOfCores <- cores
registerDoParallel(cores = nrOfCores)
message(paste('\n Registered number of cores:\n',nrOfCores,'\n'))


 Registered number of cores:
 8 




In [35]:
##################################
#                                
#          MODELING      
#                                
##################################

# helper functions
source(file.path(func_path, '94_evaluate.R'))
source(file.path(func_path, '95_fairness_metrics.R'))
source(file.path(func_path, '96_emp_summary.R'))
source(file.path(func_path, '97_caret_settings.R'))
source(file.path(func_path, '98_param_grids.R'))
source(file.path(func_path, '99_compute_profit.R'))

# modeling loop
for (method in c('RW', 'DI')) {
  for (fold in seq(0, 4)) {

    #---- PREPARATIONS ----
    
    # feedback
    print('----------------------------------------')
    print(paste0('METHOD: ', method, ' | FOLD: ', fold))
    print('----------------------------------------')      
      for (di_repair_level in all_di_repair_level) {
        # read data
        if (method == 'DI') {
          dtest  <- read.csv(file.path(data_path, 'prepared', paste0(data, '_', fold, '_pre_test_',  method, '_', di_repair_level, '.csv')))
          dval   <- read.csv(file.path(data_path, 'prepared', paste0(data, '_', fold, '_pre_valid_', method, '_', di_repair_level, '.csv')))
          dtrain <- read.csv(file.path(data_path, 'prepared', paste0(data, '_', fold, '_pre_train_', method, '_', di_repair_level, '.csv')))
        }else{
          dtest  <- read.csv(file.path(data_path, 'prepared', paste0(data, '_', fold, '_pre_test_',  method, '.csv')))
          dval   <- read.csv(file.path(data_path, 'prepared', paste0(data, '_', fold, '_pre_valid_', method, '.csv')))
          dtrain <- read.csv(file.path(data_path, 'prepared', paste0(data, '_', fold, '_pre_train_', method, '.csv')))
        }

        # load original test data
        dtest_unscaled <- read.csv(file.path(data_path, 'prepared', paste0(data, '_orig_test.csv')))
        dtest_unscaled <- subset(dtest_unscaled, select = c(CREDIT_AMNT, AGE, TARGET))

        # load original train data
        dtrain_unscaled <- read.csv(file.path(data_path, 'prepared', paste0(data, '_orig_', fold, '_train.csv')))
        dtrain_unscaled <- subset(dtrain_unscaled, select = c(CREDIT_AMNT, AGE, TARGET))

        # factor encoding
        dtrain$TARGET         <- as.factor(ifelse(dtrain$TARGET == 1,         'Good', 'Bad'))
        dval$TARGET           <- as.factor(ifelse(dval$TARGET   == 1,         'Good', 'Bad'))
        dtest$TARGET          <- as.factor(ifelse(dtest$TARGET  == 1,         'Good', 'Bad'))
        dtest_unscaled$TARGET <- as.factor(ifelse(dtest_unscaled$TARGET == 1, 'Good', 'Bad'))
        dtrain$AGE            <- as.factor(ifelse(dtrain$AGE == 1,            'Old',  'Young'))
        dval$AGE              <- as.factor(ifelse(dval$AGE   == 1,            'Old',  'Young'))
        dtest$AGE             <- as.factor(ifelse(dtest$AGE  == 1,            'Old',  'Young'))
        dtest_unscaled$AGE    <- as.factor(ifelse(dtest_unscaled$AGE == 1,    'Old',  'Young'))


        #---- TRAINING ----

        # grid search params
        source(file.path(func_path, '97_caret_settings.R'))
        source(file.path(func_path, '98_param_grids.R'))

        # train models and save result to model.'name'
        for (m in model.names) {
          print(paste0('-- ', m, '...'))
          grid <- get(paste('param.', m, sep = ''))
          args.train <- list(TARGET ~ .,
                             data      = dtrain,
                             method    = m,
                             tuneGrid  = grid,
                             metric    = 'EMP', 
                             trControl = model.control)
          args.model <- c(args.train, get(paste('args.', m, sep = '')))
          assign(paste('model.', m, sep = ''), do.call(caret::train, args.model))
          print(paste('-- model', m, 'finished training:', Sys.time(), sep = ' '))
        }

        # clean up
        for (m in model.names) {
          rm(list = c(paste0('args.', m), paste0('param.', m)))
        }
        gc()
        rm(args.model, args.train, model.control)


        #---- THRESHOLDING ----

        # Find optimal cutoff based on validation set
        for (m in model.names){
          pred <- predict(get(paste('model.', m, sep = '')), newdata = dval, type = 'prob')$Good
          EMP  <- empCreditScoring(scores = pred, classes = dval$TARGET)
          assign(paste0('cutoff.', m), EMP$EMPCfrac)
          assign(paste0('EMP.',    m), EMP$EMPC)
        }


        #---- TESTING ----

        # save image
        if (method == 'DI') {
          save.image(file.path(res_path, 'intermediate', paste0('IMAGE_PRE_', data, '_', method, '_', fold, '_', di_repair_level, '.Rdata')))
        }else{
          save.image(file.path(res_path, 'intermediate', paste0('IMAGE_PRE_', data, '_', method, '_', fold, '.Rdata')))
        }

        # reload helper functions
        source(file.path(func_path, '94_evaluate.R'))
        source(file.path(func_path, '95_fairness_metrics.R'))
        source(file.path(func_path, '96_emp_summary.R'))
        source(file.path(func_path, '97_caret_settings.R'))
        source(file.path(func_path, '98_param_grids.R'))
        source(file.path(func_path, '99_compute_profit.R'))

        # assess test results
        test_results <- NULL
        for (m in model.names) {

          # extract preds and scores
          pred         <- predict(get(paste0('model.', m)), newdata = dtest, type = 'prob')$Good
          cutoff       <- quantile(pred, get(paste0('cutoff.', m)))
          cutoff_label <- sapply(pred, function(x) ifelse(x > cutoff, 'Good', 'Bad'))

          # evaluation
          res <- evaluate(class_preds = cutoff_label, 
                          score_preds = pred,
                          targets     = dtest$TARGET, 
                          amounts     = dtest$CREDIT_AMNT * max(dtrain_unscaled$CREDIT_AMNT),
                          age         = dtest$AGE,
                          r           = 0.2644)
          test_results <- cbind(test_results, res)
        }

        # save results
        colnames(test_results) <- c(model.names)
        if (method == 'DI') {
          write.csv(test_results, file.path(res_path, 'intermediate', paste0(data, '_', fold, '_', method, '_', di_repair_level, '_results.csv')), row.names = T)
        }else{
          write.csv(test_results, file.path(res_path, 'final', paste0(data, '_', fold, '_', method, '_results.csv')), row.names = T)
        }
    }
  }
}

# close cluster
stopImplicitCluster()

[1] "----------------------------------------"
[1] "METHOD: RW | FOLD: 0"
[1] "----------------------------------------"
[1] "-- glm..."
Aggregating results
Fitting final model on full training set
[1] "-- model glm finished training: 2023-03-21 14:36:28"
[1] "-- rf..."
Aggregating results
Selecting tuning parameters
Fitting mtry = 15 on full training set
[1] "-- model rf finished training: 2023-03-21 14:39:11"
[1] "-- xgbTree..."
Aggregating results
Selecting tuning parameters
Fitting nrounds = 1000, max_depth = 10, eta = 0.1, gamma = 0, colsample_bytree = 0.5, min_child_weight = 0.5, subsample = 1 on full training set
[1] "-- model xgbTree finished training: 2023-03-21 14:55:03"
[1] "-- nnet..."
Aggregating results
Selecting tuning parameters
Fitting size = 5, decay = 1.5 on full training set
[1] "-- model nnet finished training: 2023-03-21 14:55:48"


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases



[1] "-- glm..."
Aggregating results
Fitting final model on full training set
[1] "-- model glm finished training: 2023-03-21 14:56:07"
[1] "-- rf..."
Aggregating results
Selecting tuning parameters
Fitting mtry = 15 on full training set
[1] "-- model rf finished training: 2023-03-21 14:58:46"
[1] "-- xgbTree..."
Aggregating results
Selecting tuning parameters
Fitting nrounds = 1000, max_depth = 10, eta = 0.1, gamma = 0, colsample_bytree = 1, min_child_weight = 0.5, subsample = 0.5 on full training set
[1] "-- model xgbTree finished training: 2023-03-21 15:14:50"
[1] "-- nnet..."
Aggregating results
Selecting tuning parameters
Fitting size = 5, decay = 1 on full training set
[1] "-- model nnet finished training: 2023-03-21 15:15:36"


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases



[1] "-- glm..."
Aggregating results
Fitting final model on full training set
[1] "-- model glm finished training: 2023-03-21 15:15:55"
[1] "-- rf..."
Aggregating results
Selecting tuning parameters
Fitting mtry = 15 on full training set
[1] "-- model rf finished training: 2023-03-21 15:18:41"
[1] "-- xgbTree..."
Aggregating results
Selecting tuning parameters
Fitting nrounds = 1000, max_depth = 10, eta = 0.1, gamma = 0, colsample_bytree = 0.5, min_child_weight = 0.5, subsample = 1 on full training set
[1] "-- model xgbTree finished training: 2023-03-21 15:34:27"
[1] "-- nnet..."
Aggregating results
Selecting tuning parameters
Fitting size = 5, decay = 0.1 on full training set
[1] "-- model nnet finished training: 2023-03-21 15:35:18"


...

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases



[1] "-- glm..."
Aggregating results
Fitting final model on full training set
[1] "-- model glm finished training: 2023-03-22 09:22:06"
[1] "-- rf..."
Aggregating results
Selecting tuning parameters
Fitting mtry = 15 on full training set
[1] "-- model rf finished training: 2023-03-22 09:24:43"
[1] "-- xgbTree..."
Aggregating results
Selecting tuning parameters
Fitting nrounds = 100, max_depth = 5, eta = 0.1, gamma = 0, colsample_bytree = 0.5, min_child_weight = 3, subsample = 1 on full training set
[1] "-- model xgbTree finished training: 2023-03-22 09:39:40"
[1] "-- nnet..."
Aggregating results
Selecting tuning parameters
Fitting size = 5, decay = 1.5 on full training set
[1] "-- model nnet finished training: 2023-03-22 09:40:24"


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases



[1] "-- glm..."
Aggregating results
Fitting final model on full training set
[1] "-- model glm finished training: 2023-03-22 09:40:41"
[1] "-- rf..."
Aggregating results
Selecting tuning parameters
Fitting mtry = 10 on full training set
[1] "-- model rf finished training: 2023-03-22 09:43:15"
[1] "-- xgbTree..."
Aggregating results
Selecting tuning parameters
Fitting nrounds = 100, max_depth = 5, eta = 0.1, gamma = 0, colsample_bytree = 0.5, min_child_weight = 1, subsample = 1 on full training set
[1] "-- model xgbTree finished training: 2023-03-22 09:57:47"
[1] "-- nnet..."
Aggregating results
Selecting tuning parameters
Fitting size = 5, decay = 1 on full training set
[1] "-- model nnet finished training: 2023-03-22 09:58:31"


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

Setting levels: control = Bad, case = Good

Setting direction: controls < cases

